In [1]:
!pip install transformers -U -q
import torch
import tensorflow as tf

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 16.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 23.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 33.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 37.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 23.7 MB/s eta 0:00:00


In [2]:
! pip install sentencepiece

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 15.4 MB/s eta 0:00:00


In [3]:
!pip freeze | grep transformers

transformers==4.35.0


In [5]:
from transformers import MarianMTModel, MarianTokenizer
import nltk

# Download the part-of-speech tagger data
nltk.download('averaged_perceptron_tagger')

# Initialize the MarianMT model and tokenizer
model_name = 'Helsinki-NLP/opus-mt-en-hi'
tokenizer = MarianTokenizer.from_pretrained(model_name)
model = MarianMTModel.from_pretrained(model_name)

# Function to check if a word is a noun
def is_noun(text):
    # Perform part-of-speech tagging on the input text
    ans = nltk.pos_tag([text])
    val = ans[0][1]

    # Check if the tag represents a noun
    if val in ('NN', 'NNS', 'NNPS', 'NNP'):
        return True
    else:
        return False

# Sample input sentences
input_text = [
    "Definitely share your feedback in the comment section.",
    "So even if it's a big video, I will clearly mention all the products.",
    "I was waiting for my bag."
]

# Function to translate and preserve nouns in the input text
def translate_and_preserve_nouns(input_text):
    translated_sentences = []

    for text in input_text:
        # Tokenize and generate translation for the input sentence
        model_inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True)
        generated_tokens = model.generate(**model_inputs)
        translation = tokenizer.decode(generated_tokens[0], skip_special_tokens=True)

        # Split the input sentence into words and identify nouns
        words = text.split()
        noun_list = [word for word in words if is_noun(word)]

        for noun in noun_list:
            noun = noun.replace(',', '')
            # Tokenize and generate translation for each noun
            model_inputs_noun = tokenizer(noun, return_tensors="pt", padding=True, truncation=True)
            generated_tokens_noun = model.generate(**model_inputs_noun)
            noun_translation = tokenizer.decode(generated_tokens_noun[0], skip_special_tokens=True)

            # Replace the translated noun in the overall translation
            translation = translation.replace(noun_translation, noun)

        translated_sentences.append(translation)

    return translated_sentences

# Translate and preserve nouns in the input sentences
translated_sentences = translate_and_preserve_nouns(input_text)





[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
/usr/local/lib/python3.10/dist-packages/transformers/models/marian/tokenization_marian.py:197: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


In [6]:

# Print the translated sentences
for sentence in translated_sentences:
    print(sentence)

comment खण्ड में अपनी प्रतिक्रिया को निश्‍चित ही share करें ।
तो यह एक बड़ा video है, तो भी मैं स्पष्ट रूप से सभी उत्पादों का mention करेंगे।
मैं अपने बैग के लिए इंतजार कर रहा था.
